In [2]:
import json
from objectbox import (
    Entity,
    Id,
    String,
    Store,
    Box,
    Float32Vector,
    HnswIndex,
    VectorDistanceType,
    
)
from pydantic import BaseModel
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from pathlib import Path
import yaml

/home/a.durynin/Projects/Python/TabletopMagnatAll/TabletopMagnat/.venv/lib64/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
rules_chunks_pth = Path("../data/iki/chunks.json")
rules_chunks: list[dict] = json.loads(rules_chunks_pth.read_text())
term_ner_pth = Path("../data/iki/terms.json")
term_ner: list[dict] = json.loads(term_ner_pth.read_text())

In [4]:
rules_chunks[0]

{'id': 'iki_001',
 'type': 'main_title',
 'section': 'title',
 'content': 'Iki — игра о мастерах эпохи Эдо. Погрузитесь в мир Японии 1603–1868 гг. В районе Нихонбаси находится самый оживлённый рынок города. На главной улице района расположились самые разные лавки и торговцы, приглашающие покупателей заглянуть к ним на огонёк. Неподалёку от главной улицы находится рынок морепродуктов, где рыбаки сбывают свой улов — оттуда слышатся голоса покупателей и торгующее рыбой. Ремесленники и мастера живут в домах-нагая вокруг главной улицы. Их умения и товары помогают простым людям в Эдо жить лучше. Говорят, что в те времена в Эдо было от семисот до восьмисот разных профессий, и многие из этих уникальных умений стали картами в игре Iki: мастера по изготовлению вееров, фонарей, татами, художники укиё-э, уличные торговцы, владельцы прилавков с суши и темпурой, гейши и актёры театра Кабуки.',
 'req_term': ['Iki', 'Эдо', 'Нихонбаси', 'мастера', 'professions'],
 'scenario': 'Вступительное описание ми

In [5]:
term_ner[0]

{'id': 'term_iki_001',
 'name': 'Ояката',
 'slug': 'oyakata',
 'kind': 'TERM',
 'path': 'iki/game_components/figures/oyakata',
 'group': 'game_components',
 'definition': 'Фигурка игрока, которой он перемещается по главной улице Нихонбаси, заключает сделки в магазинах и активирует персонажей в комнатах над магазинами.',
 'extra': {'used_in': ['фаза B', 'заключение сделок', 'движение'],
  'symbols': ['🧑\u200d🦰']}}

In [6]:
for rule in rules_chunks:
    tags = " ".join(
        [
            f"#section:{rule['section']}",
            f"#type:{rule['type']}",
        ]
    )
    rule["scenario"] = f"{tags}\n---\n{rule["scenario"]}"
    rule["req_term"] = yaml.safe_dump(rule["req_term"], allow_unicode=True)

In [7]:
print(rules_chunks[2]["content"])

Состав игры: 1 двустороннее поле и 4 двусторонних планшета игрока (для 2 игроков и 3-4 игроков). Карты: 4 стартовых карты персонажей, 56 карт персонажей (14 карт для каждого сезона): Весна 🌸, Лето ☀️, Осень 🍂, Зима ❄️. Деревянные элементы: 4 фигурки Ояката 🧑‍🦰 (фиолетовая, синяя, жёлтая, красная), 4 фигурки Икизама 🧑‍🦱 (фиолетовая, синяя, жёлтая, красная), 16 фигурок Кобун 🧑‍🦳 (по 4 фиолетовых, 4 синих, 4 жёлтых, 4 красных), 4 маркера для подсчёта очков Ики 🌸 (по одному цвету: розовый, фиолетовый, жёлтый, красный), 4 маркера огнеупорности 🔥 (по одному цвету: розовый, фиолетовый, жёлтый, красный). Жетоны и монеты: 20 заготовок древесины 🪵, 30 мешков риса 🍚, 1 маркер календаря 📅, 8 жетонов трубок (по 2 на каждый сезон) 🚬, 8 жетонов табака (по 2 на каждый сезон) 🍃, 36 монет номиналом 1 мон 🪙, 20 монет номиналом 4 мона 🪙, 28 жетонов сандалий 👟, 12 жетонов Кобан (золотых слитков) 🏦, 8 жетонов рыб (по 2 на каждый сезон) 🐟, 4 жетона «30/60» очков Ики (для каждого игрока) 🌸, 4 жетона пожара 🔥.

In [8]:
for term in term_ner:
    tags =  f"#group:{term["group"]}"
    term["content"] = f"{tags}\n---\n{term["name"]}"
    term["extra"] = yaml.safe_dump(term["extra"] if "extra" in term else [], allow_unicode=True)

In [9]:
term_ner[0]

{'id': 'term_iki_001',
 'name': 'Ояката',
 'slug': 'oyakata',
 'kind': 'TERM',
 'path': 'iki/game_components/figures/oyakata',
 'group': 'game_components',
 'definition': 'Фигурка игрока, которой он перемещается по главной улице Нихонбаси, заключает сделки в магазинах и активирует персонажей в комнатах над магазинами.',
 'extra': 'symbols:\n- 🧑\u200d🦰\nused_in:\n- фаза B\n- заключение сделок\n- движение\n',
 'content': '#group:game_components\n---\nОяката'}

In [10]:
from objectbox import Int16


@Entity()
class Rule:
    id = Id
    internal_id = String
    content = String
    section = String
    game= String
    req_term = String
    scenario = String
    priority = Int16
    zone = String
    vector = Float32Vector(
        index=HnswIndex(dimensions=768, distance_type=VectorDistanceType.EUCLIDEAN)
    )
   
@Entity()
class Terminology:
    id = Id
    internal_id = String
    game = String
    content = String
    name = String
    slug = String
    kind = String
    path = String
    group = String
    definition = String
    extra = String
    vector = Float32Vector(
        index = HnswIndex(dimensions=768)
    )

@Entity()
class Game:
    id = Id
    name = String
    latin_name = String
    vector = Float32Vector(
        index = HnswIndex(dimensions=768)
    )

In [11]:
store = Store(directory="../db")
rules_box = Box(store=store, entity=Rule)
term_ner_box = Box(store=store, entity=Terminology)
game_box = Box(store=store, entity=Game)

---

In [12]:
model = SentenceTransformer("../model").to("cpu")

In [13]:
box_objects = [
    Rule(
        internal_id=obj["id"],
        content=obj["content"],
        section=obj["section"],
        game="Iki",
        req_term=obj["req_term"],
        scenario=obj["scenario"],
        priority=obj["priority"],
        zone=obj["zone"],
        vector=model.encode(obj["scenario"]).tolist(),
    )
    for obj in tqdm(rules_chunks)
]

100%|██████████| 52/52 [00:10<00:00,  4.94it/s]


In [14]:
box_terms = [
    Terminology(
        internal_id=obj["id"],
        content=obj["content"],
        name=obj["name"],
        slug=obj["slug"],
        kind=obj["kind"],
        path=obj["path"],
        group=obj["group"],
        definition=obj["definition"],
        extra=obj["extra"],
        vector=model.encode(obj["content"]).tolist(),
    )
    for obj in tqdm(term_ner)
]

100%|██████████| 32/32 [00:06<00:00,  5.05it/s]


In [15]:
box_games = [
    Game(
        name="Подземелье и пёсики",
        latin_name="Podzemelie i pesiki",
        vector=model.encode("Подземелье и пёсики").tolist(),
    ),
    Game(
        name="Iki",
        latin_name="Iki",
        vector=model.encode("Iki").tolist(),
    )
]

In [16]:
rules_box.put(box_objects)
term_ner_box.put(box_terms)
game_box.put(box_games)

In [17]:
store.close()